# Use a federated learning strategy

Welcome to the next part of the federated learning tutorial. In previous parts of this tutorial, we introduced federated learning with PyTorch and Flower ([part 1](https://flower.dev/docs/framework/tutorial-get-started-with-flower-pytorch.html)).

In this notebook, we'll begin to customize the federated learning system we built in the introductory notebook (again, using [Flower](https://flower.dev/) and [PyTorch](https://pytorch.org/)).

> [Star Flower on GitHub](https://github.com/adap/flower) ⭐️ and join the Flower community on Slack to connect, ask questions, and get help: [Join Slack](https://flower.dev/join-slack) 🌼 We'd love to hear from you in the `#introductions` channel! And if anything is unclear, head over to the `#questions` channel.

Let's move beyond FedAvg with Flower strategies!

## Preparation

Before we begin with the actual code, let's make sure that we have everything we need.

### Installing dependencies

First, we install the necessary packages:

In [1]:
!pip install -q flwr[simulation] torch torchvision

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [2]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.0.1+cu117 and Flower 1.4.0


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

### Data loading

Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap everything in their own `DataLoader`. We introduce a new parameter `num_clients` which allows us to call `load_datasets` with different numbers of clients.

In [3]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


### Model training/evaluation

Let's continue with the usual model definition (including `set_parameters` and `get_parameters`), training and test functions:

In [4]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

### Flower client

To implement the Flower client, we (again) create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`. Here, we also pass the `cid` to the client and use it log additional details:

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

## Strategy customization

So far, everything should look familiar if you've worked through the introductory notebook. With that, we're ready to introduce a number of new features. 

### Server-side parameter **initialization**

Flower, by default, initializes the global model by asking one random client for the initial parameters. In many cases, we want more control over parameter initialization though. Flower therefore allows you to directly pass the initial parameters to the Strategy:

In [6]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-08-25 16:41:03,232 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-08-25 16:41:05,518	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-08-25 16:41:07,143 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 97757731021.0, 'object_store_memory': 46181884723.0, 'GPU': 8.0, 'node:172.28.177.173': 1.0, 'node:__internal_head__': 1.0, 'CPU': 80.0}
INFO flwr 2023-08-25 16:41:07,144 | server.py:86 | Initializing global parameters
INFO flwr 2023-08-25 16:41:07,145 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-08-25 16:41:07,147 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-08-25 16:41:07,147 | server.py:101 | FL starting
DEBUG flwr 2023-08-25 16:41:07,149 | server.py:218 | fit_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1371782) [Client 4] fit, config: {}


DEBUG flwr 2023-08-25 16:41:14,238 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-08-25 16:41:14,249 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-08-25 16:41:14,250 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1371782) Epoch 1: train loss 0.06582393497228622, accuracy 0.2288888888888889


DEBUG flwr 2023-08-25 16:41:17,506 | server.py:182 | evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-08-25 16:41:17,508 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-08-25 16:41:17,510 | server.py:218 | fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-08-25 16:41:30,967 | server.py:232 | fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-08-25 16:41:30,975 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1371768) Epoch 1: train loss 0.05554109439253807, accuracy 0.34933333333333333
(launch_and_fit pid=1371783) Epoch 1: train loss 0.055733997374773026, accuracy 0.3402222222222222


DEBUG flwr 2023-08-25 16:41:34,143 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-08-25 16:41:34,145 | server.py:147 | FL finished in 26.99680959060788
INFO flwr 2023-08-25 16:41:34,148 | app.py:218 | app_fit: losses_distributed [(1, 0.06367969091733296), (2, 0.058633947372436525), (3, 0.05535255058606466)]
INFO flwr 2023-08-25 16:41:34,149 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-08-25 16:41:34,150 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-08-25 16:41:34,151 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2023-08-25 16:41:34,152 | app.py:222 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=1371768) [Client 9] evaluate, config: {}
(launch_and_fit pid=1371782) [Client 5] fit, config: {}
(launch_and_evaluate pid=1371783) [Client 1] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06367969091733296
	round 2: 0.058633947372436525
	round 3: 0.05535255058606466

Passing `initial_parameters` to the `FedAvg` strategy prevents Flower from asking one of the clients for the initial parameters. If we look closely, we can see that the logs do not show any calls to the `FlowerClient.get_parameters` method.

### Starting with a customized strategy

We've seen the function `start_simulation` before. It accepts a number of arguments, amongst them the `client_fn` used to create `FlowerClient` instances, the number of clients to simulate `num_clients`, the number of rounds `num_rounds`, and the strategy.

The strategy encapsulates the federated learning approach/algorithm, for example, `FedAvg` or `FedAdagrad`. Let's try to use a different strategy this time:

In [7]:
# Create FedAdam strategy
strategy = fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-08-25 16:42:17,926 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-08-25 16:42:22,124	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-08-25 16:42:23,781 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 97712644301.0, 'node:__internal_head__': 1.0, 'node:172.28.177.173': 1.0, 'GPU': 8.0, 'object_store_memory': 46162561843.0, 'CPU': 80.0}
INFO flwr 2023-08-25 16:42:23,783 | server.py:86 | Initializing global parameters
INFO flwr 2023-08-25 16:42:23,784 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-08-25 16:42:23,784 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-08-25 16:42:23,785 | server.py:101 | FL starting
DEBUG flwr 2023-08-25 16:42:23,787 | server.py:218 | fit_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1386927) [Client 9] fit, config: {}


DEBUG flwr 2023-08-25 16:42:30,801 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-08-25 16:42:30,811 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-08-25 16:42:30,817 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1386868) Epoch 1: train loss 0.06543184071779251, accuracy 0.21288888888888888
(launch_and_fit pid=1386924) [Client 0] fit, config: {} [repeated 2x across cluster]
(launch_and_evaluate pid=1386927) [Client 2] evaluate, config: {}


DEBUG flwr 2023-08-25 16:42:34,026 | server.py:182 | evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-08-25 16:42:34,027 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-08-25 16:42:34,029 | server.py:218 | fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-08-25 16:42:47,644 | server.py:232 | fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-08-25 16:42:47,657 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1386924) Epoch 1: train loss 0.07578238099813461, accuracy 0.16066666666666668 [repeated 2x across cluster]


DEBUG flwr 2023-08-25 16:42:50,853 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-08-25 16:42:50,854 | server.py:147 | FL finished in 27.067959655076265
INFO flwr 2023-08-25 16:42:50,856 | app.py:218 | app_fit: losses_distributed [(1, 4.751413950602213), (2, 0.2475512479146322), (3, 0.10965212011337279)]
INFO flwr 2023-08-25 16:42:50,857 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-08-25 16:42:50,858 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-08-25 16:42:50,858 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2023-08-25 16:42:50,859 | app.py:222 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=1386927) [Client 2] evaluate, config: {}
(launch_and_fit pid=1386868) [Client 2] fit, config: {}
(launch_and_evaluate pid=1386868) [Client 9] evaluate, config: {}


History (loss, distributed):
	round 1: 4.751413950602213
	round 2: 0.2475512479146322
	round 3: 0.10965212011337279

## Server-side parameter **evaluation**

Flower can evaluate the aggregated model on the server-side or on the client-side. Client-side and server-side evaluation are similar in some ways, but different in others.

**Centralized Evaluation** (or *server-side evaluation*) is conceptually simple: it works the same way that evaluation in centralized machine learning does. If there is a server-side dataset that can be used for evaluation purposes, then that's great. We can evaluate the newly aggregated model after each round of training without having to send the model to clients. We're also fortunate in the sense that our entire evaluation dataset is available at all times.

**Federated Evaluation** (or *client-side evaluation*) is more complex, but also more powerful: it doesn't require a centralized dataset and allows us to evaluate models over a larger set of data, which often yields more realistic evaluation results. In fact, many scenarios require us to use **Federated Evaluation** if we want to get representative evaluation results at all. But this power comes at a cost: once we start to evaluate on the client side, we should be aware that our evaluation dataset can change over consecutive rounds of learning if those clients are not always available. Moreover, the dataset held by each client can also change over consecutive rounds. This can lead to evaluation results that are not stable, so even if we would not change the model, we'd see our evaluation results fluctuate over consecutive rounds.

We've seen how federated evaluation works on the client side (i.e., by implementing the `evaluate` method in `FlowerClient`). Now let's see how we can evaluate aggregated model parameters on the server-side:

In [8]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [9]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-08-25 16:50:34,843 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-08-25 16:50:39,057	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-08-25 16:50:40,522 | app.py:180 | Flower VCE: Ray initialized with resources: {'GPU': 8.0, 'object_store_memory': 46143342182.0, 'node:172.28.177.173': 1.0, 'node:__internal_head__': 1.0, 'memory': 97667798426.0, 'CPU': 80.0}
INFO flwr 2023-08-25 16:50:40,523 | server.py:86 | Initializing global parameters
INFO flwr 2023-08-25 16:50:40,524 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-08-25 16:50:40,525 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-08-25 16:50:40,687 | server.py:91 | initial parameters (loss, other metrics): 0.07360826683044433, {'accuracy': 0.1}
INFO flwr 2023-08-25 16:50:40,687 | server.py:101 | FL starting
DEBUG flwr 2023-08-25 16:50:40,690 | server.py:218 | fit_round 1: strategy sampled 3 clients 

Server-side evaluation loss 0.07360826683044433 / accuracy 0.1
(launch_and_fit pid=1474666) [Client 4] fit, config: {}
(launch_and_fit pid=1474632) Epoch 1: train loss 0.06457281857728958, accuracy 0.23044444444444445


DEBUG flwr 2023-08-25 16:50:47,752 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-08-25 16:50:47,761 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-08-25 16:50:47,919 | server.py:119 | fit progress: (1, 0.06191800975799561, {'accuracy': 0.292}, 7.229074642062187)
DEBUG flwr 2023-08-25 16:50:47,920 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.06191800975799561 / accuracy 0.292


DEBUG flwr 2023-08-25 16:50:51,131 | server.py:182 | evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-08-25 16:50:51,132 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-08-25 16:50:51,133 | server.py:218 | fit_round 2: strategy sampled 3 clients (out of 10)


...

INFO flwr 2023-08-25 16:51:05,525 | server.py:119 | fit progress: (3, 0.05336381769180298, {'accuracy': 0.372}, 24.83516550064087)
DEBUG flwr 2023-08-25 16:51:05,526 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.05336381769180298 / accuracy 0.372


DEBUG flwr 2023-08-25 16:51:08,596 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-08-25 16:51:08,598 | server.py:147 | FL finished in 27.907622922211885
INFO flwr 2023-08-25 16:51:08,599 | app.py:218 | app_fit: losses_distributed [(1, 0.06120289699236552), (2, 0.05528086352348327), (3, 0.05136912751197815)]
INFO flwr 2023-08-25 16:51:08,600 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-08-25 16:51:08,601 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-08-25 16:51:08,602 | app.py:221 | app_fit: losses_centralized [(0, 0.07360826683044433), (1, 0.06191800975799561), (2, 0.05564793944358826), (3, 0.05336381769180298)]
INFO flwr 2023-08-25 16:51:08,603 | app.py:222 | app_fit: metrics_centralized {'accuracy': [(0, 0.1), (1, 0.292), (2, 0.336), (3, 0.372)]}


(launch_and_evaluate pid=1474666) [Client 2] evaluate, config: {}
(launch_and_fit pid=1474632) [Client 1] fit, config: {}
(launch_and_evaluate pid=1474649) [Client 8] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06120289699236552
	round 2: 0.05528086352348327
	round 3: 0.05136912751197815
History (loss, centralized):
	round 0: 0.07360826683044433
	round 1: 0.06191800975799561
	round 2: 0.05564793944358826
	round 3: 0.05336381769180298
History (metrics, centralized):
{'accuracy': [(0, 0.1), (1, 0.292), (2, 0.336), (3, 0.372)]}

## Sending/receiving arbitrary values to/from clients

In some situations, we want to configure client-side execution (training, evaluation) from the server-side. One example for that is the server asking the clients to train for a certain number of local epochs. Flower provides a way to send configuration values from the server to the clients using a dictionary. Let's look at an example where the clients receive values from the server through the `config` parameter in `fit` (`config` is also available in `evaluate`). The `fit` method receives the configuration dictionary through the `config` parameter and can then read values from this dictionary. In this example, it reads `server_round` and `local_epochs` and uses those values to improve the logging and configure the number of local training epochs:

In [10]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

So how can we  send this config dictionary from server to clients? The built-in Flower Strategies provide way to do this, and it works similarly to the way server-side evaluation works. We provide a function to the strategy, and the strategy calls this function for every round of federated learning:

In [11]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

Next, we'll just pass this function to the FedAvg strategy before starting the simulation:

In [13]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-08-25 17:24:06,859 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-08-25 17:24:11,051	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-08-25 17:24:12,465 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 45554904268.0, 'memory': 96294776628.0, 'node:__internal_head__': 1.0, 'node:172.28.177.173': 1.0, 'CPU': 80.0, 'GPU': 8.0}
INFO flwr 2023-08-25 17:24:12,467 | server.py:86 | Initializing global parameters
INFO flwr 2023-08-25 17:24:12,468 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-08-25 17:24:12,469 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-08-25 17:24:12,913 | server.py:91 | initial parameters (loss, other metrics): 0.07375138854980469, {'accuracy': 0.098}
INFO flwr 2023-08-25 17:24:12,913 | server.py:101 | FL starting
DEBUG flwr 2023-08-25 17:24:12,914 | server.py:218 | fit_round 1: strategy sampled 3 client

Server-side evaluation loss 0.07375138854980469 / accuracy 0.098
(launch_and_fit pid=1848459) [Client 4, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


DEBUG flwr 2023-08-25 17:24:19,990 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-08-25 17:24:19,999 | fedavg.py:243 | No fit_metrics_aggregation_fn provided


(launch_and_fit pid=1848459) Epoch 1: train loss 0.06459751725196838, accuracy 0.2288888888888889


INFO flwr 2023-08-25 17:24:20,261 | server.py:119 | fit progress: (1, 0.06259999442100525, {'accuracy': 0.28}, 7.347835049033165)
DEBUG flwr 2023-08-25 17:24:20,262 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-08-25 17:24:41,571 | server.py:232 | fit_round 3 received 3 results and 0 failures
INFO flwr 2023-08-25 17:24:41,721 | server.py:119 | fit progress: (3, 0.050082776069641115, {'accuracy': 0.432}, 28.807085759937763)
DEBUG flwr 2023-08-25 17:24:41,721 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.050082776069641115 / accuracy 0.432


DEBUG flwr 2023-08-25 17:24:44,851 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-08-25 17:24:44,853 | server.py:147 | FL finished in 31.938989464193583
INFO flwr 2023-08-25 17:24:44,853 | app.py:218 | app_fit: losses_distributed [(1, 0.06234831833839417), (2, 0.05407735363642375), (3, 0.049298333565394085)]
INFO flwr 2023-08-25 17:24:44,854 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-08-25 17:24:44,854 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-08-25 17:24:44,855 | app.py:221 | app_fit: losses_centralized [(0, 0.07375138854980469), (1, 0.06259999442100525), (2, 0.05397287464141846), (3, 0.050082776069641115)]
INFO flwr 2023-08-25 17:24:44,855 | app.py:222 | app_fit: metrics_centralized {'accuracy': [(0, 0.098), (1, 0.28), (2, 0.376), (3, 0.432)]}


(launch_and_evaluate pid=1848459) [Client 6] evaluate, config: {}
(launch_and_fit pid=1848465) [Client 2, round 3] fit, config: {'server_round': 3, 'local_epochs': 2} [repeated 2x across cluster]
(launch_and_fit pid=1848465) Epoch 2: train loss 0.04974992573261261, accuracy 0.4137777777777778 [repeated 4x across cluster]


History (loss, distributed):
	round 1: 0.06234831833839417
	round 2: 0.05407735363642375
	round 3: 0.049298333565394085
History (loss, centralized):
	round 0: 0.07375138854980469
	round 1: 0.06259999442100525
	round 2: 0.05397287464141846
	round 3: 0.050082776069641115
History (metrics, centralized):
{'accuracy': [(0, 0.098), (1, 0.28), (2, 0.376), (3, 0.432)]}

As we can see, the client logs now include the current round of federated learning (which they read from the `config` dictionary). We can also configure local training to run for one epoch during the first and second round of federated learning, and then for two epochs during the third round.

Clients can also return arbitrary values to the server. To do so, they return a dictionary from `fit` and/or `evaluate`. We have seen and used this concept throughout this notebook without mentioning it explicitly: our `FlowerClient` returns a dictionary containing a custom key/value pair as the third return value in `evaluate`.

## Scaling federated learning

As a last step in this notebook, let's see how we can use Flower to experiment with a large number of clients.

In [14]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


We now have 1000 partitions, each holding 45 training and 5 validation examples. Given that the number of training examples on each client is quite small, we should probably train the model a bit longer, so we configure the clients to perform 3 local training epochs. We should also adjust the fraction of clients selected for training during each round (we don't want all 1000 clients participating in every round), so we adjust `fraction_fit` to `0.05`, which means that only 5% of available clients (so 50 clients) will be selected for training each round:


In [15]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config


strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-08-25 17:25:22,266 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-08-25 17:25:26,198	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-08-25 17:25:27,664 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 96304751616.0, 'CPU': 80.0, 'object_store_memory': 45559179264.0, 'GPU': 8.0, 'node:172.28.177.173': 1.0, 'node:__internal_head__': 1.0}
INFO flwr 2023-08-25 17:25:27,670 | server.py:86 | Initializing global parameters
INFO flwr 2023-08-25 17:25:27,672 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-08-25 17:25:27,673 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-08-25 17:25:27,674 | server.py:101 | FL starting
DEBUG flwr 2023-08-25 17:25:27,675 | server.py:218 | fit_round 1: strategy sampled 25 clients (out of 1000)


(launch_and_fit pid=1863383) [Client 420, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1863383) Epoch 1: train loss 0.1040804535150528, accuracy 0.06666666666666667
(launch_and_fit pid=1863383) Epoch 2: train loss 0.1027059331536293, accuracy 0.08888888888888889
(launch_and_fit pid=1863383) Epoch 3: train loss 0.10209744423627853, accuracy 0.15555555555555556
(launch_and_fit pid=1863383) [Client 841, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1863383) Epoch 1: train loss 0.10233889520168304, accuracy 0.1111111111111111
(launch_and_fit pid=1863383) Epoch 2: train loss 0.10215666145086288, accuracy 0.1111111111111111
(launch_and_fit pid=1863383) Epoch 3: train loss 0.10230378806591034, accuracy 0.1111111111111111
(launch_and_fit pid=1863383) [Client 664, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1863383) Epoch 1: train loss 0.10269245505332947, accuracy 0.044444444444444446
(l

DEBUG flwr 2023-08-25 17:25:57,164 | server.py:232 | fit_round 1 received 25 results and 0 failures
WARNING flwr 2023-08-25 17:25:57,206 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-08-25 17:25:57,207 | server.py:168 | evaluate_round 1: strategy sampled 50 clients (out of 1000)


(launch_and_fit pid=1863426) [Client 868, round 1] fit, config: {'server_round': 1, 'local_epochs': 3} [repeated 22x across cluster]
(launch_and_fit pid=1863379) Epoch 3: train loss 0.101410873234272, accuracy 0.2 [repeated 63x across cluster]
(launch_and_evaluate pid=1863426) [Client 747] evaluate, config: {}
(launch_and_fit pid=1863425) Epoch 3: train loss 0.10180016607046127, accuracy 0.1111111111111111 [repeated 3x across cluster]
(launch_and_evaluate pid=1863426) [Client 151] evaluate, config: {}
(launch_and_evaluate pid=1863419) [Client 500] evaluate, config: {}
(launch_and_evaluate pid=1863419) [Client 67] evaluate, config: {}
(launch_and_evaluate pid=1863419) [Client 954] evaluate, config: {}
(launch_and_evaluate pid=1863419) [Client 239] evaluate, config: {} [repeated 3x across cluster]
(launch_and_evaluate pid=1863379) [Client 216] evaluate, config: {} [repeated 4x across cluster]
(launch_and_evaluate pid=1863379) [Client 926] evaluate, config: {} [repeated 2x across cluster]

DEBUG flwr 2023-08-25 17:26:49,672 | server.py:182 | evaluate_round 1 received 50 results and 0 failures
WARNING flwr 2023-08-25 17:26:49,673 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-08-25 17:26:49,675 | server.py:218 | fit_round 2: strategy sampled 25 clients (out of 1000)


...

DEBUG flwr 2023-08-25 17:28:35,313 | server.py:232 | fit_round 3 received 25 results and 0 failures
DEBUG flwr 2023-08-25 17:28:35,350 | server.py:168 | evaluate_round 3: strategy sampled 50 clients (out of 1000)


(launch_and_fit pid=1863426) Epoch 3: train loss 0.09815005958080292, accuracy 0.15555555555555556 [repeated 66x across cluster]
(launch_and_fit pid=1863426) [Client 860, round 3] fit, config: {'server_round': 3, 'local_epochs': 3} [repeated 15x across cluster]
(launch_and_evaluate pid=1863337) [Client 54] evaluate, config: {}
(launch_and_evaluate pid=1863426) [Client 340] evaluate, config: {} [repeated 6x across cluster]
(launch_and_evaluate pid=1863419) [Client 771] evaluate, config: {} [repeated 7x across cluster]
(launch_and_evaluate pid=1863426) [Client 846] evaluate, config: {} [repeated 5x across cluster]


DEBUG flwr 2023-08-25 17:29:27,982 | server.py:182 | evaluate_round 3 received 50 results and 0 failures
INFO flwr 2023-08-25 17:29:27,983 | server.py:147 | FL finished in 240.30791148915887
INFO flwr 2023-08-25 17:29:27,983 | app.py:218 | app_fit: losses_distributed [(1, 0.4604914598464967), (2, 0.4577830133438109), (3, 0.4534457941055298)]
INFO flwr 2023-08-25 17:29:27,984 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-08-25 17:29:27,985 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-08-25 17:29:27,985 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2023-08-25 17:29:27,985 | app.py:222 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.4604914598464967
	round 2: 0.4577830133438109
	round 3: 0.4534457941055298

(launch_and_evaluate pid=1863436) [Client 289] evaluate, config: {}


## Recap

In this notebook, we've seen how we can gradually enhance our system by customizing the strategy, initializing parameters on the server side, choosing a different strategy, and evaluating models on the server-side. That's quite a bit of flexibility with so little code, right?

In the later sections, we've seen how we can communicate arbitrary values between server and clients to fully customize client-side execution. With that capability, we built a large-scale Federated Learning simulation using the Flower Virtual Client Engine and ran an experiment involving 1000 clients in the same workload - all in a Jupyter Notebook!

## Next steps

Before you continue, make sure to join the Flower community on Slack: [Join Slack](https://flower.dev/join-slack/)

There's a dedicated `#questions` channel if you need help, but we'd also love to hear who you are in `#introductions`!

The [Flower Federated Learning Tutorial - Part 3](https://flower.dev/docs/framework/tutorial-build-a-strategy-from-scratch-pytorch.html) shows how to build a fully custom `Strategy` from scratch.